In [ ]:
import os
import cv2
from tkinter import *
from tkinter import filedialog
from shutil import copyfile
from PIL import Image, ImageTk
from cv2 import VideoWriter, VideoWriter_fourcc
from ipynb.fs.full.DataCollectionUtil import *
from ipynb.fs.full.GraphicalUserInterface import *
from ipynb.fs.full.DigitalImageProcessingUtil import *
from ipynb.fs.full.ColourSpaceConversionUtil import *
from ipynb.fs.full.ApplicationUtil import *
from ipynb.fs.full.VideoProcessingUtil import *
from ipynb.fs.full.RoadUsersDetectionAndTracking import *
from ipynb.fs.full.LicensePlateRecognition import *


# Traffic enforcement camera constants
CAMERA_STREAM_FOLDER='../camera_stream/'
CAMERA_INPUT_VIDEO_FILE_NAME='Netcam.mp4'
PROCESSED_OUTPUT_VIDEOS_FOLDER='../processed_videos/'
PROCESSED_OUTPUT_VIDEO_FILENAME1=os.path.join(PROCESSED_OUTPUT_VIDEOS_FOLDER,'1_Pavement_Markings.mp4')
OUTPUT_VIDEO_WIDTH=1920
OUTPUT_VIDEO_HEIGHT=1080
OUTPUT_VIDEO_FPS=30.26
OUTPUT_VIDEO_FOURCC=VideoWriter_fourcc(*'MP42')

# Dataset preparation constants
TRAFFIC_LIGHT_DATASET_FOLDER='../traffic_light_dataset/'

# GUI constants
GUI_UPLOADED_FILE_BACKUP="../file_backup/"

#Stop line determination functionality constants
RESIZE_WIDTH=1280
RESIZE_HEIGHT=720
RESIZE_PIXEL_AREA_RELATION=cv2.INTER_AREA

# Pavement marking functionality constants
ROI_VERTEX_1=(0, 640)
ROI_VERTEX_2=(900, 400)
ROI_VERTEX_3=(1920,700)
ROI_VERTEX_4=(1920,1080)
ROI_VERTEX_5=(0,1080)


# Global variables
stopLineCoordinates=[]


# Defining functions
def getStopLineCoordinates(event, x, y, flags, params):
    global stopLineCoordinates
    
    # Verifying if a left button click event was triggered or not
    if event == cv2.EVENT_LBUTTONDOWN:
        
        # Printing the x and y coordinates on the traffic enforcement camera image
        font = cv2.cv2.FONT_HERSHEY_COMPLEX_SMALL
        cv2.putText(uploadedTrafficCameraImage, str(x) + ',' + str(y), (x,y), font, 1, (255, 255, 255), 2)
        
        # Displaying the same traffic camera image with the coordinates
        cv2.imshow('Traffic Enforcement Camera image', uploadedTrafficCameraImage)
        
        # Adding the selected co-ordinates to the list
        point=str(x)+','+str(y)
        stopLineCoordinates.append(point)
    
    # Verifying if a right button click event was triggered or not    
    if event==cv2.EVENT_RBUTTONDOWN:
        
        # Printing the x and y coordinates on the traffic enforcement camera image
        font = cv2.FONT_HERSHEY_COMPLEX_SMALL
        b = img[y, x, 0]
        g = img[y, x, 1]
        r = img[y, x, 2]
        cv2.putText(uploadedTrafficCameraImage, str(b) + ',' + str(g) + ',' + str(r), (x,y), font, 1, (255, 255, 255), 2)
        
        # Displaying the same traffic camera image with the coordinates
        cv2.imshow('Traffic Enforcement Camera image', uploadedTrafficCameraImage)
        
        # Adding the selected co-ordinates to the list
        point=str(x)+','+str(y)
        stopLineCoordinates.append(point)
        
        return

def getPavementMarkings(frame):
    # Reading the original, colour image as a frame
    trafficCameraImage=frame.copy()
    
    # Applying probabilistic Hough Line Transform to the frame
    edgeDetectedTrafficCameraImage=probabilisticHoughLineTransform(trafficCameraImage)    
    
    # Assigning the vertices for the region of interest in the frame
    region_of_interest_vertices = [ROI_VERTEX_1,ROI_VERTEX_2,ROI_VERTEX_3,ROI_VERTEX_4,ROI_VERTEX_5]
    
    # Cropping the frame based on the region of interest
    imageAfterRegionOfInterest = region_of_interest(edgeDetectedTrafficCameraImage,np.array([region_of_interest_vertices], np.int32),)
    
    # Re-applying probabilistic Hough Line Transform to the cropped frame
    pavementMarkings = cv2.HoughLinesP(imageAfterRegionOfInterest,1,np.pi/180,180,minLineLength=100,maxLineGap=50)
    
    # Drawing the pavement markings on the original frame
    imageWithPavementMarkings = drawTheStopLine(frame, pavementMarkings)
    
    return imageWithPavementMarkings



# The top-level script with the highest priority
if __name__=="__main__":
    
        
    # Dataset preparation
    # (To be run only for the first time)
    # completionFlag=getFramesFromVideo(cameraStreamFolder,TRAFFIC_LIGHT_DATASET_FOLDER)
    # if(completionFlag):
        # print('Main script ---- The images for the dataset has been successfully generated!')

    
    # Launching the application's graphical user interface only to rectify the stop line coordinates   
    # launchGUIWindow()
    
    #Determining the stop line co-ordinates
    # Reading the uploaded traffic enforcement camera image
    uploadedTrafficCameraImage = cv2.imread(GUI_UPLOADED_FILE_BACKUP+'Cam1.png', 1)
    
    # Resizing the uploaded image while maintaining aspect ratio
    uploadedTrafficCameraImage=maintainAspectRatioWhileResizing(uploadedTrafficCameraImage,RESIZE_WIDTH,RESIZE_HEIGHT,RESIZE_PIXEL_AREA_RELATION)
    
    # Displaying the resized traffic enforcement camera image
    cv2.imshow('Traffic Enforcement Camera image', uploadedTrafficCameraImage)
    
    # Setting the mouse call back function on the resized traffic enforcement camera image by invoking the getStopLineCoordinates() function
    cv2.setMouseCallback('Traffic Enforcement Camera image', getStopLineCoordinates)
    
    # Waiting for a key press so as to exit
    cv2.waitKey(0)
    
    # Destroying all the windows
    cv2.destroyAllWindows()
    
    # Preparing the video writer for generating the pavement markings video
    pavementMarkingsVideo = VideoWriter(PROCESSED_OUTPUT_VIDEO_FILENAME1, OUTPUT_VIDEO_FOURCC, float(OUTPUT_VIDEO_FPS), (OUTPUT_VIDEO_WIDTH, OUTPUT_VIDEO_HEIGHT))
    
    print('###### Module 1 - Stop line detection begins ######')
    print('Main_Script ---- Generating the pavement markings video . . .')
    
    # Obtaining the stream from the traffic enforcement camera
    videoCapture = VideoProcessingUtil(os.path.join(CAMERA_STREAM_FOLDER, CAMERA_INPUT_VIDEO_FILE_NAME)).start()
    time.sleep(1.0)
    
    while (videoCapture.next()):
        frame = videoCapture.readQueue()
        if np.shape(frame) == ():
            break
        else:
            try:
                height, width, _ = frame.shape
            except AttributeError:
                print('Main_Script ---- Attribute error in the current frame while processing.')
                continue
            frameInRGB=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frameWithLines = getPavementMarkings(frameInRGB)
            outputFrameInRGB=cv2.cvtColor(frameWithLines, cv2.COLOR_BGR2RGB)
            pavementMarkingsVideo.write(outputFrameInRGB)
    
    # Release the resources of the pavement markings video and the red light violations video
    pavementMarkingsVideo.release()
    videoCapture.stop()
    
    # Waiting for a key press so as to exit
    cv2.waitKey(0)
    
    # Destroying all the windows
    cv2.destroyAllWindows()
    
    print('Main_Script ---- The pavement markings video has been successfully generated.')
    print('###### Module 1 - Stop line detection ends ######')
    
    detectAndTrackRoadUsers('../processed_videos/1_Pavement_Markings.mp4', 500, 800, 500, 1920)
    
    print('###### Module 4 - License plate recognition begins ######')
    getLicensePlateNumberFromFrame('../file_backup/Red_Light_Violation_Frame.jpeg')
    print('###### Module 4 - License plate recognition ends ######')
    
    print('Main_Script ---- Red light violations have been determined by the system!')